In [ ]:
# Use if you run the notebook on Google colab
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
!pip install mglearn

# 2: $k$-Nearest Neighbours

> If two things are similar, the thought of one will tend to trigger the thought of the other <br>
-- Aristotle

## Imports

In [ ]:
import sys
import os
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("/content/drive/MyDrive/50603/code")
os.chdir('/content/drive/MyDrive/50603')

import ipywidgets as widgets
import mglearn
from IPython.display import display
from ipywidgets import interact, interactive
from plotting_functions import *
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate, train_test_split
from utils import *

%matplotlib inline

pd.set_option("display.max_colwidth", 200)
import warnings

warnings.filterwarnings("ignore")

<br><br>

### Quick recap

- Why do we split the data?
- What are the advantages of cross-validation?
- What is overfitting?
- What's the fundamental trade-off in supervised machine learning?
- Types of data for our purposes
  - Categorical:
    - Nominal (sometimes just called *categorical*!), Ordinal
  - Numerical:
    - Discrete, Continuous

## Learning outcomes

From this lecture, you will be able to

- explain the notion of similarity-based algorithms;
- broadly describe how $k$-NNs use distances;
- discuss the effect of using a small/large value of the hyperparameter $k$ when using the $k$-NN algorithm;
- describe the problem of curse of dimensionality;

## Motivation and distances [[video](https://youtu.be/hCa3EXEUmQk)]

### Instance-based models

- Suppose you are given the following training examples with corresponding labels and are asked to label a given test example.

<!-- <img src='./img/knn-motivation.png' width="1000"> -->



In [ ]:
from IPython.display import Image, display
p = 'img/knn-motivation.png'
display(Image(filename=p, width=1000))

[source](https://vipl.ict.ac.cn/en/database.php)

- An intuitive way to classify the test example is by finding the most "similar" example(s) from the training set and using that label for the test example.

### General idea of $k$-nearest neighbours algorithm

- Consider the following toy dataset with two classes.
    - blue circles $\rightarrow$ class 0
    - red triangles $\rightarrow$ class 1
    - green stars $\rightarrow$ test examples

In [ ]:
X, y = mglearn.datasets.make_forge()
X_test = np.array([[8.2, 3.7], [9.9, 3.2], [11.2, 0.5]])

In [ ]:
# plot_train_test_points is defined in code/plotting_functions.py

plot_train_test_points(X, y, X_test)

- Given a new data point, predict the class of the data point by finding the "closest" data point in the training set, i.e., by finding its "nearest neighbour" or majority vote of nearest neighbours.

In [ ]:
# plot_knn_clf is defined in code/plotting_functions.py

def f(n_neighbors):
    return plot_knn_clf(X, y, X_test, n_neighbors=n_neighbors)

In [ ]:
interactive(
    f,
    n_neighbors=widgets.IntSlider(min=1, max=7, step=2, value=1),
)

### **Geometric view** of tabular data and dimensions

- To understand instance-based algorithms it's useful to think of <u>data as points</u> in a high dimensional space.
- Our `X` represents the the problem in terms of relevant **features** ($d$) with one dimension for each **feature** (column).
- Examples are **points in a $d$-dimensional space**.

How many dimensions (features) are there in the cities data?

In [ ]:
cities_df = pd.read_csv("data/canada_usa_cities.csv")
X_cities = cities_df[["longitude", "latitude"]]
y_cities = cities_df["country"]

In [ ]:
mglearn.discrete_scatter(X_cities.iloc[:, 0], X_cities.iloc[:, 1], y_cities)
plt.xlabel("longitude")
plt.ylabel("latitude");

### Dimensions in ML problems

In ML, usually we deal with high dimensional problems where examples are hard to visualize.  

- $d \approx 20$ is considered low dimensional
- $d \approx 1000$ is considered medium dimensional
- $d \approx 100,000$ is considered high dimensional

### Feature vectors

**Feature vector** is composed of feature values associated with an example.

They correspond to <u>rows</u> of our dataframes.

Some example feature vectors are shown below.

In [ ]:
print("\nAn example feature vector from the cities dataset: %s"
      % (X_cities.iloc[0].to_numpy()))

### **Similarity** between examples

Let's take 2 points (two feature vectors) from the cities dataset.

In [ ]:
two_cities = X_cities.sample(2, random_state=120)
two_cities

In [ ]:
# also recall that y was the country for each city
y_cities.unique()

The two sampled points are shown as big black circles.

In [ ]:
mglearn.discrete_scatter(
    X_cities.iloc[:, 0], X_cities.iloc[:, 1], y_cities, s=8, alpha=0.3
)
mglearn.discrete_scatter(
    two_cities.iloc[:, 0], two_cities.iloc[:, 1], markers="o", c="k", s=18
);

### Distance between feature vectors

- For the `two_cities` at the two big circles, what is the _distance_ between them?
- A common way to calculate the distance between vectors is calculating the **Euclidean distance**.
- The euclidean distance
  - in one dimension between $u$ and $v$:
  $$distance(u, v) = \sqrt{(u - v)^2} = | u - v |$$

  - in two dimensions between $u = <u_1, u_2>$ and $v = <v_1, v_2>$:
  $$distance(u, v) = \sqrt{(u_1 - v_1)^2 + (u_2 - v_2)^2}$$

  - generally, in higher dimensions between vectors <br>
  $u = <u_1, u_2, \dots, u_n>$ and $v = <v_1, v_2, \dots, v_n>$ is defined as:

$$distance(u, v) = \sqrt{\sum_{i =1}^{n} (u_i - v_i)^2}$$


### Euclidean distance

In [ ]:
two_cities

- Subtract the two cities
- Square the difference
- Sum them up
- Take the square root

In [ ]:
# Subtract the two cities
print("Subtract the cities: \n%s\n"
      % (two_cities.iloc[1] - two_cities.iloc[0]))

# Squared sum of the difference
print("Sum of squares: %0.4f\n"
      % (np.sum((two_cities.iloc[1] - two_cities.iloc[0]) ** 2)))

# Take the square root
print("Euclidean distance between cities: %0.4f"
      % (np.sqrt(np.sum((two_cities.iloc[1] - two_cities.iloc[0]) ** 2))))

In [ ]:
two_cities

In [ ]:
# Euclidean distance using sklearn
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(two_cities)

Note: `scikit-learn` supports a number of other [distance metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html).


### Finding the nearest neighbour

- Let's look at distances from all cities to all other cities

In [ ]:
dists = euclidean_distances(X_cities)
np.fill_diagonal(dists, np.inf)  # why is this needed?
print("All distances: ", dists.shape)
pd.DataFrame(dists)

Let's look at the distances between City 0 and some other cities.

In [ ]:
print("Feature vector for city 0: \n%s\n" % (X_cities.iloc[0]))
print("Distances from city 0 to the first 5 cities: \n%s\n" % (dists[0,:5]))
# We can find the closest city with `np.argmin`:
print("The closest city from city 0 is: %d \nwith feature vector: \n%s"
      % (np.argmin(dists[0]), X_cities.iloc[np.argmin(dists[0])]))

So, we **managed to find out the closest city to City 0, which is City 81**.

### Question

- Why did we set the diagonal entries to infinity before finding the closest city?

### Finding the distances to a query point

We can also find the distances to a **new "test" or "query" city**:

In [ ]:
# Let's find a city that's closest to a query city
query_point = [[-80, 25]]

dists = euclidean_distances(X_cities, query_point)
dists[0:10]

In [ ]:
print("The query point %s is closest to the city with index %d \n"
      "and the distance between them is: %0.4f"
      % (query_point, np.argmin(dists), dists[np.argmin(dists)]))

In [ ]:
# See the closest city (72) among some other cities with thir distances to query point
X_cities.join(pd.DataFrame(dists, columns=['dist'])).head(np.argmin(dists) + 3).tail()

<br><br>

## $k$-Nearest Neighbours ($k$-NNs) [[video](https://youtu.be/bENDqXKJLmg)]

In [ ]:
small_cities = cities_df.sample(30, random_state=90)
one_city = small_cities.sample(1, random_state=44)
# get all of small_cities excluding one_city:
small_train_df = pd.concat([small_cities, one_city]).drop_duplicates(keep=False)

In [ ]:
X_small_cities = small_train_df[["longitude", "latitude"]].to_numpy()
y_small_cities = small_train_df["country"].to_numpy()
test_point = one_city[["longitude", "latitude"]].to_numpy()

In [ ]:
plot_train_test_points(
    X_small_cities,
    y_small_cities,
    test_point,
    class_names=["Canada", "USA"],
    test_format="circle",
)

- Given a new data point, predict the class of the data point by finding the "closest" data point in the training set, i.e., by finding its "nearest neighbour" or majority vote of nearest neighbours.

Suppose we want to predict the class of the black point.  
- An intuitive way to do this is predict the same label as the "closest" point ($k = 1$) (1-nearest neighbour)
- We would predict a target of **USA** in this case.

In [ ]:
plot_knn_clf(
    X_small_cities,
    y_small_cities,
    test_point,
    n_neighbors=1,
    class_names=["Canada", "USA"],
    test_format="circle",
)

How about using $k > 1$ to get a more robust estimate?
- For example, we could also use the 3 closest points (*k* = 3) and let them **vote** on the correct class.  
- The **Canada** class would win in this case.

In [ ]:
plot_knn_clf(
    X_small_cities,
    y_small_cities,
    test_point,
    n_neighbors=3,
    class_names=["Canada", "USA"],
    test_format="circle",
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k_values = [1, 3, 5]

for k in k_values:
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_small_cities, y_small_cities)
    print("Prediction of the black dot with %d neighbours: %s"
          % (k, neigh.predict(test_point)))

### Choosing `n_neighbors`

- The **primary hyperparameter** of the model is `n_neighbors` ($k$) which decides how many neighbours should vote during prediction?
- What happens when we play around with `n_neighbors`?
- Are we more likely to overfit with a low `n_neighbors` or a high `n_neighbors`?
- Let's examine the effect of the hyperparameter on our cities data.

In [ ]:
X = cities_df.drop(columns=["country"])
y = cities_df["country"]

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=123
)

In [ ]:
k = 1
knn1 = KNeighborsClassifier(n_neighbors=k)
scores = cross_validate(knn1, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

In [ ]:
k = 100
knn100 = KNeighborsClassifier(n_neighbors=k)
scores = cross_validate(knn100, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

In [ ]:
def f(n_neighbors=1):
    results = {}
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    scores = cross_validate(knn, X_train, y_train, return_train_score=True)
    results["n_neighbours"] = [n_neighbors]
    results["mean_train_score"] = [round(scores["train_score"].mean(), 3)]
    results["mean_valid_score"] = [round(scores["test_score"].mean(), 3)]
    print(pd.DataFrame(results))


interactive(
    f,
    n_neighbors=widgets.IntSlider(min=1, max=101, step=10, value=1),
)

In [ ]:
plot_knn_decision_boundaries(X_train, y_train, k_values=[1, 11, 100])

### How to choose `n_neighbors`?

- `n_neighbors` is a hyperparameter
- We can use ***hyperparameter optimization*** to choose `n_neighbors`.

In [ ]:
results_dict = {
    "n_neighbors": [],
    "mean_train_error": [],
    "mean_cv_error": [],
    "std_cv_score": [],
    "std_train_score": [],
}
param_grid = {"n_neighbors": np.arange(1, 50, 5)}

for k in param_grid["n_neighbors"]:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, X_train, y_train, return_train_score=True)
    results_dict["n_neighbors"].append(k)

    results_dict["mean_cv_error"].append(1-np.mean(scores["test_score"]))
    results_dict["mean_train_error"].append(1-np.mean(scores["train_score"]))
    results_dict["std_cv_score"].append(scores["test_score"].std())
    results_dict["std_train_score"].append(scores["train_score"].std())

results_df = pd.DataFrame(results_dict)

In [ ]:
results_df = results_df.set_index("n_neighbors")
results_df

In [ ]:
results_df[["mean_train_error", "mean_cv_error"]].plot().invert_xaxis();

In [ ]:
best_n_neighbours = results_df.idxmin()["mean_cv_error"]
best_n_neighbours

Let's try our best model on test data.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_n_neighbours)
knn.fit(X_train, y_train)
print("Test accuracy: %0.3f" % (knn.score(X_test, y_test)))

<br><br>

## More on $k$-NNs [[video](https://youtu.be/IRGbqi5S9gQ)]

### Other useful arguments of `KNeighborsClassifier`

- `weights` $\rightarrow$ When predicting label, you can assign higher weight to the examples which are closer to the query example.  
- Exercise for you: Play around with this argument. Do you get a better validation score?

### Pros of $k$-NNs for supervised learning

- Easy to understand, interpret.
- Simple hyperparameter $k$ (`n_neighbors`) controlling the fundamental tradeoff.
- Can learn very complex functions given enough data.
- **Lazy learning**: Takes no time to `fit`

### Cons of $k$-NNs for supervised learning

- Can be potentially VERY **slow during prediction** time, especially when the training set is very large.
- Often not that great test accuracy compared to the modern approaches.
- It does not work well on datasets with many features or where most feature values are 0 most of the time (sparse datasets).    

***Important***
> For regular $k$-NN for supervised learning (not with sparse matrices), you should scale your features. We'll be looking into it soon.

### Parametric vs non parametric

- You might see a lot of definitions of these terms.
- A simple way to think about this is:
    - *For $n$ samples, do you need to store at least $O(n)$* worth of stuff to make predictions? If so, it's non-parametric.
- **Non-parametric example**: $k$-NN is a classic example of non-parametric models.     

***Note***
> $\mathcal{O}(n)$ is referred to as big $\mathcal{O}$ notation. It tells you how fast an algorithm is or how much storage space it requires. For example, in simple terms, if you have $n$ samples and you need to store them all you can say that the algorithm requires $\mathcal{O}(n)$ worth of stuff.

### Curse of dimensionality

- Affects all learners but especially bad for nearest-neighbour.
- $k$-NN usually works well when the number of dimensions $d$ is small but things fall apart quickly as $d$ goes up.
- If there are **many irrelevant attributes, $k$-NN is hopelessly confused** because all of them contribute to finding similarity between examples.
- With enough irrelevant attributes the accidental similarity swamps out meaningful similarity and $k$-NN is no better than random guessing.  

In [ ]:
from sklearn.datasets import make_classification

nfeats_accuracy = {"nfeats": [], "dummy_valid_accuracy": [], "KNN_valid_accuracy": []}
for n_feats in range(4, 2000, 100):
    X, y = make_classification(n_samples=2000, n_features=n_feats, n_classes=2)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=123
    )
    dummy = DummyClassifier(strategy="most_frequent")
    dummy_scores = cross_validate(dummy, X_train, y_train, return_train_score=True)

    knn = KNeighborsClassifier()
    scores = cross_validate(knn, X_train, y_train, return_train_score=True)
    nfeats_accuracy["nfeats"].append(n_feats)
    nfeats_accuracy["KNN_valid_accuracy"].append(np.mean(scores["test_score"]))
    nfeats_accuracy["dummy_valid_accuracy"].append(np.mean(dummy_scores["test_score"]))

In [ ]:
pd.DataFrame(nfeats_accuracy)

In [ ]:
pd.DataFrame(nfeats_accuracy).set_index("nfeats").plot();

<br><br>

## Summary

- We have KNNs as new supervised learning techniques in our toolbox.
- These are instance-based learners and the idea is to assign nearby points the same label.
- Can be used for classification or regression.